In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install kaggle

In [2]:
%cd /content/gdrive/MyDrive/MalignantTextClassificationDataset

/content/gdrive/MyDrive/MalignantTextClassificationDataset


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/MalignantTextClassificationDataset"

In [ ]:
!kaggle datasets download -d surekharamireddy/malignant-comment-classification
!unzip malignant-comment-classification.zip

In [16]:
import numpy as np
import sys
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

sys.setrecursionlimit(200000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def creatDF(file):
  df = pd.read_csv(file)
  data = []
  for row in df.iterrows():
    try:
      if int(row[1]['malignant'])==0 and int(row[1]['highly_malignant'])==0 and int(row[1]['rude'])==0 and int(row[1]['threat'])==0 and int(row[1]['abuse'])==0 and int(row[1]['loathe'])==0:
        data.append([row[1]['comment_text'], 0])
      else:
        data.append([row[1]['comment_text'], 1])
    except:
      KeyError
  return pd.DataFrame(data, columns = ['text', 'isBad'])

In [14]:
data = creatDF('train.csv').sample(frac=1).reset_index(drop=True)

Y = data['isBad']
X = data.drop('isBad',axis=1)

In [17]:
import re
from tqdm.notebook import tqdm
ps = PorterStemmer()

corpus = []
length = len(X)

for i in tqdm(range(0, length)):
    review = re.sub('[^a-zA-Z]', ' ', X['text'][i]).lower().split()
    review = ' '.join([ps.stem(word) for word in review if not word in stopwords.words('english')])
    corpus.append(review)

  0%|          | 0/159571 [00:00<?, ?it/s]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=3000)
X = cv.fit_transform(corpus).toarray()

In [20]:
import pickle

pickle.dump(cv, open('cv_transfrom.pkl', 'wb'))

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [22]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(alpha=0.3)
model.fit(X_train, Y_train)

MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)

In [24]:
pickle.dump(model, open("Final_Model.pkl", 'wb'))